# 分布式抓取：多处理

**通过分布式抓取和解析加快抓取速度。我将抓取 [我的网站](https://mofanpy.com/)，但在本地服务器(http://127.0.0.1:4000/)中以消除不同的下载速度。该测试在时间测量方面更准确。您可以改用(https://mofanpy.com/)，因为您无法访问(http://127.0.0.1:4000/)。**

**我们将抓取我网站中的所有网页，并重新查找每个页面的标题和网址。**

In [1]:
import multiprocessing as mp
import time
from urllib.request import urlopen, urljoin
from bs4 import BeautifulSoup
import re

# base_url = "http://127.0.0.1:4000/"
base_url = 'http://mofanpy.com/'

# 不要过度抓取网站，否则您可能永远不会再访问
if base_url != "http://127.0.0.1:4000/":
    restricted_crawl = True
else:
    restricted_crawl = False

**创建一个抓取函数以并行打开 url。**

In [2]:
def crawl(url):
    """
    爬取指定URL的网页内容
    
    参数:
        url (str): 要爬取的网页URL地址
        
    返回:
        str: 网页的HTML内容（已解码为字符串）
    """
    # 使用urlopen发送HTTP请求获取网页内容
    response = urlopen(url)
    # time.sleep(0.1)             # 稍微延迟一下，避免请求过于频繁（被注释掉）
    # 读取响应内容并解码为字符串（假设是UTF-8编码）
    return response.read().decode()

**创建一个解析函数来并行查找我们需要的所有结果**

In [3]:

def parse(html):
    """
    解析HTML内容，提取标题、页面链接和当前页面URL
    
    参数:
        html (str): 要解析的HTML内容字符串
        
    返回:
        tuple: 包含三个元素的元组
            - title (str): 页面标题（从<h1>标签提取）
            - page_urls (set): 页面中所有符合特定模式的链接集合（已去重）
            - url (str): 当前页面的规范URL（从og:url元标签提取）
    """
    # 使用BeautifulSoup解析HTML文档，使用lxml解析器
    soup = BeautifulSoup(html, 'lxml')
    
    # 查找所有href属性符合正则表达式'^/.+?/$'的<a>标签
    # 正则解释：^表示开始，/表示斜杠，.+?表示一个或多个任意字符（非贪婪），/$表示以斜杠结尾
    # 这个模式匹配所有以斜杠开头、以斜杠结尾的相对路径URL
    urls = soup.find_all('a', {"href": re.compile('^/.+?/$')})
    
    # 提取<h1>标签的文本内容，并去除前后空白字符
    title = soup.find('h1').get_text().strip()
    
    # 使用集合推导式处理找到的URL：
    # 1. 使用urljoin将相对路径转换为绝对URL
    # 2. 使用set()自动去重
    page_urls = set([urljoin(base_url, url['href']) for url in urls])
    
    # 从Open Graph协议的og:url元标签中提取当前页面的规范URL
    url = soup.find('meta', {'property': "og:url"})['content']
    
    # 返回提取的标题、页面链接集合和当前页面URL
    return title, page_urls, url

## 正常方式
**请不要使用多进程，测试速度。首先，在 Python 集合中设置我们已见过和未见过的 URL。**

In [4]:

# 初始化两个集合：
# unseen: 存储待爬取的URL，初始包含基础URL
# seen: 存储已爬取的URL，初始为空
unseen = set([base_url,])
seen = set()

# 初始化计数器和计时器
# count: 用于记录已处理的页面数量
# t1: 记录爬虫开始时间，用于计算总耗时
count, t1 = 1, time.time()

# 主循环：当unseen集合不为空时继续爬取
while len(unseen) != 0:                 # 仍然获得一些 URL 访问
    # 检查是否设置了限制爬取的条件，并且已爬取页面数量超过20
    if restricted_crawl and len(seen) > 20:
            break  # 如果满足条件，则退出循环，停止爬取
    
    print('\nDistributed Crawling...')  # 打印提示信息，表示开始分布式爬取
    # 使用列表推导式，对unseen集合中的每个URL调用crawl函数，获取HTML内容
    htmls = [crawl(url) for url in unseen]
    
    print('\nDistributed Parsing...')   # 打印提示信息，表示开始分布式解析
    # 使用列表推导式，对每个HTML内容调用parse函数，解析出标题、页面链接和当前URL
    results = [parse(html) for html in htmls]
    
    print('\nAnalysing...')             # 打印提示信息，表示开始分析结果
    
    # 将unseen集合中的所有URL添加到seen集合中，表示这些URL已经被访问
    seen.update(unseen)         # 已查看爬取内容
    
    # 清空unseen集合，准备存储新发现的URL
    unseen.clear()              # 没有未被看见的
    
    # 遍历解析结果，处理每个页面
    for title, page_urls, url in results:
        # 打印页面序号、标题和URL
        print(count, title, url)
        count += 1  # 增加页面计数器
        
        # 从当前页面解析出的所有URL中，减去已经访问过的URL，得到新的待爬取URL
        # 并将这些新URL添加到unseen集合中
        unseen.update(page_urls - seen)     # get new url to crawl

# 计算并打印总耗时
# 检查并打印请求的URL，确保其格式正确
print("请求的URL:", url)



Distributed Crawling...


HTTPError: HTTP Error 400: Bad Request

## multiprocessing
**Create a process pool and scrape parallelly.**

In [5]:
unseen = set([base_url,])
seen = set()

pool = mp.Pool(4)                       
count, t1 = 1, time.time()
while len(unseen) != 0:                 # still get some url to visit
    if restricted_crawl and len(seen) > 20:
            break
    print('\nDistributed Crawling...')
    crawl_jobs = [pool.apply_async(crawl, args=(url,)) for url in unseen]
    htmls = [j.get() for j in crawl_jobs]                                       # request connection

    print('\nDistributed Parsing...')
    parse_jobs = [pool.apply_async(parse, args=(html,)) for html in htmls]
    results = [j.get() for j in parse_jobs]                                     # parse html

    print('\nAnalysing...')
    seen.update(unseen)         # seen the crawled
    unseen.clear()              # nothing unseen

    for title, page_urls, url in results:
        print(count, title, url)
        count += 1
        unseen.update(page_urls - seen)     # get new url to crawl
print('Total time: %.1f s' % (time.time()-t1, ))    # 16 s !!!


Distributed Crawling...
